In [ ]:
%matplotlib inline
import os
import struct
import torch
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import cv2

In [ ]:
def fftImg(Img):
    Img = np.array(Img)
    fImg = np.fft.fft2(Img)
    fshiftImg = np.fft.fftshift(fImg)
    AbsImg = np.log(np.abs(fshiftImg))
    AngImg = np.angle(fshiftImg)
    return AbsImg, AngImg

In [ ]:
def cropimg(Img):
    w,h,c = Img.shape
    return Img[w/2-112:w/2+112,h/2-112:h/2+112].transpose(2,0,1)

In [ ]:
def gammaImg(image, gamma):
    invGamma = 1.0 / gamma
    table = np.array([((i / 255.0) ** invGamma) * 255
        for i in np.arange(0, 256)]).astype("uint8")
    return cv2.LUT(image, table)

In [ ]:
def compressImg(img,q):
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), q]
    result, encimg = cv2.imencode('.jpg', img, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg

In [ ]:
def interpolImg(Img, factor):
    w,h,c = Img.shape
    factorx = factor
    factory = factor
    if factor*h<224:
        factorx = 1
    if factor*w<224:
        factory = 1
    return cv2.resize(Img,None,fx=factorx,fy=factory, interpolation = cv2.INTER_CUBIC)

In [ ]:
listing = ['Sony-NEX-7',
 'Motorola-X',
 'HTC-1-M7',
 'Samsung-Galaxy-Note3',
 'Motorola-Droid-Maxx',
 'iPhone-4s',
 'iPhone-6',
 'LG-Nexus-5x',
 'Samsung-Galaxy-S4',
 'Motorola-Nexus-6']

In [ ]:
Datapath = 'train/'
listImg=[]
for i in range(len(listing)):
    listImg.append(os.listdir(Datapath+listing[i]))
print(listing)

In [ ]:
TrainImg = torch.ByteTensor(275*10, 3, 224, 224)
TrainL = torch.Tensor(275*10)

TrainImgs = torch.ByteTensor(275*80, 3, 224, 224)
TrainLs = torch.Tensor(275*80)

for i in range(len(listImg)):
    print(i)
    for j in range(len(listImg[i])):
        imgfilepath = Datapath+listing[i]+'/'+listImg[i][j]
        imgfile = cv2.imread(imgfilepath)
        
        TrainImg[i*275+j] = torch.from_numpy(cropimg(imgfile))
        TrainL[i*275+j] = i

        
        
        TrainImgs[i*275*8+j] = torch.from_numpy(cropimg(compressImg(imgfile,70)))
        TrainLs[i*275*8+j] = i
        
        TrainImgs[i*275*8+275+j] = torch.from_numpy(cropimg(compressImg(imgfile,90)))
        TrainLs[i*275*8+275+j] = i
        
        TrainImgs[i*275*8+275*2+j] = torch.from_numpy(cropimg(interpolImg(imgfile,0.5)))
        TrainLs[i*275*8+275*2+j] = i
        
        TrainImgs[i*275*8+275*3+j] = torch.from_numpy(cropimg(interpolImg(imgfile,0.8)))
        TrainLs[i*275*8+275*3+j] = i
        
        TrainImgs[i*275*8+275*4+j] = torch.from_numpy(cropimg(interpolImg(imgfile,1.5)))
        TrainLs[i*275*8+275*4+j] = i
        
        TrainImgs[i*275*8+275*5+j] = torch.from_numpy(cropimg(interpolImg(imgfile,2.0)))
        TrainLs[i*275*8+275*5+j] = i
        
        TrainImgs[i*275*8+275*6+j] = torch.from_numpy(cropimg(gammaImg(imgfile,0.8)))
        TrainLs[i*275*8+275*6+j] = i
        
        TrainImgs[i*275*8+275*7+j] = torch.from_numpy(cropimg(gammaImg(imgfile,1.2)))
        TrainLs[i*275*8+275*7+j] = i

In [ ]:
torch.save(TrainImg, 'data/TrainImg.pth')
torch.save(TrainL, 'data/TrainLabel.pth')

torch.save(TrainImgs, 'data/TrainImgs.pth')
torch.save(TrainLs, 'data/TrainLabels.pth')